# Sprint　ディープラーニングフレームワーク1  

＜目的＞  
- フレームワークのコードを読めるようにする
- フレームワークを習得し続けられるようになる
- 理論を知っている範囲をフレームワークで動かす

＜コードリーディング＞  
TensorFLowによって2値分類を行うサンプルコードを載せました。今回はこれをベースにして進めます。  
tf.kerasやtf.estimatorなどの高レベルAPIは使用していません。低レベルなところから見ていくことにします。

### 【問題1】スクラッチを振り返る  
ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。

（例）  
- 重みを初期化する必要があった
- エポックのループが必要だった

それらがフレームワークにおいてはどのように実装されるかを今回覚えていきましょう。

### ＜回答＞  
- 画像データの平滑化
- 各初期化手法での重み・バイアスの初期化
- 特徴量と重みの積和（forward）
- 各活性化関数による次層への出力計算（forward）
- 予測結果からの誤差の計算（backward）
- 各活性化関数による誤差計算（backward）
- 入力値との積和による重み誤差の計算（backward）
- 各最適化手法による重み・バイアスの更新（backward）
- 次層への誤差の受け渡し（backward）

＜データセットの用意＞  
以前から使用しているIrisデータセットを使用します。以下のサンプルコードではIris.csvが同じ階層にある想定です。  
目的変数はSpeciesですが、3種類ある中から以下の2種類のみを取り出して使用します。  

- versicolor
- virginica

### 【問題2】スクラッチとTensorFlowの対応を考える  
以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。  
それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。

In [1]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 0, loss : 23.3991, val_loss : 55.1257, acc : 0.750, val_acc : 0.375
Epoch 1, loss : 0.6691, val_loss : 3.6498, acc : 0.750, val_acc : 0.438
Epoch 2, loss : 0.0001, val_loss : 0.3437, acc : 1.000, val_acc : 0.875
Epoch 3, loss : 0.3750, val_loss : 0.9712, acc : 0.750, val_acc : 0.875
Epoch 4, loss : 0.0103, val_loss : 1.5831, acc : 1.000, val_acc : 0.750
Epoch 5, loss : 0.0000, val_loss : 0.3678, acc : 1.000, val_acc : 0.938
Epoch 6, loss : 0.0000, val_loss : 0.3274, acc : 1.000, val_acc : 0.875
Epoch 7, loss : 0.0000, val_loss : 0.1388, acc : 1.000, val_acc : 0.938
Epoch 8, loss : 0.0000, val_loss : 0.0001, acc : 1.000, val_acc : 1.000
Epoch 9, loss : 0.0000, val_loss : 0.0002, acc : 1.000, val_acc : 1.000
test_acc : 0.900


### ＜回答＞  
- 画像データの平滑化  
→ 今回はIrisデータのため平滑化しないが、``tf.placeholder("float", [None, n_input])``で入力の形状を定めている


- 各初期化手法での重み・バイアスの初期化  
→ ``tf.Variable(tf.random_normal([n_input, n_hidden1]))``で初期化している


- 特徴量と重みの積和（forward）  
→ ``tf.add(tf.matmul(x, weights['w']), biases['b'])``で積和している


- 各活性化関数による次層への出力計算（forward）  
→ ``layer_1 = tf.nn.relu(layer_1)``で活性化関数を経て次層に渡す ``layer_1`` を計算し、これを引数にとって``layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])``で次層へ渡し、処理していく


- 予測結果からの誤差の計算（backward）  
→ ``loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))``で誤差計算（クロスエントロピー）している


- 各活性化関数による誤差計算（backward）
- 入力値との積和による重み誤差の計算（backward）
- 各最適化手法による重み・バイアスの更新（backward）
- 次層への誤差の受け渡し（backward）  
→ ``optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)``, ``train_op = optimizer.minimize(loss_op)``で、スクラッチでは誤差逆伝播で行っていた重みとバイアスの更新をしている


…TensorFlowでは、計算グラフ構造と初期化・活性化・最適化手法を与えれば、誤差逆伝播実装の必要なくパラメーター更新をしてくれるようだ

＜他のデータセットへの適応＞  
これまで扱ってきた小さなデータセットがいくつかあります。上記サンプルコードを書き換え、これらに対して学習・推定を行うニューラルネットワークを作成してください。

- Iris（3種類全ての目的変数を使用）
- House Prices

どのデータセットもtrain, val, testの3種類に分けて使用してください。


### 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成  
Irisデータセットのtrain.csvの中で、目的変数Speciesに含まれる3種類全てを分類できるモデルを作成してください。  
2クラスの分類と3クラス以上の分類の違いを考慮してください。それがTensorFlowでどのように書き換えられるかを公式ドキュメントなどを参考に調べてください。

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-setosa'] = 0
y[y=='Iris-versicolor'] = 1
y[y=='Iris-virginica'] = 2
y = y.astype(np.int)[:, np.newaxis]

# yをone_hotへ
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y = enc.fit_transform(y)
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# 標準化
sc = StandardScaler()
# トレーニングデータから平均・標準偏差を計算
sc.fit(X_train)
# 上記を用いて標準化
X_train_std = sc.transform(X_train)
X_val_std = sc.transform(X_val)
X_test_std = sc.transform(X_test)

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
#trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train_std, y_train, batch_size=batch_size)

# 計算グラフの構築
def example_net_1(x):
    """
    単純な3層ニューラルネットワーク(Iris 3種類)
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net_1(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss=loss_op)

# 推定結果
correct_pred = tf.equal(tf.argmax(Y, axis=1), tf.argmax(logits, axis=1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        total_loss_val = 0
        total_acc_val = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            _, loss, acc = sess.run([train_op, loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val_std, Y: y_val})
            total_loss += loss
            total_acc += acc
            total_loss_val += val_loss
            total_acc_val += val_acc
        total_loss /= total_batch
        total_acc /= total_batch
        total_loss_val /= total_batch
        total_acc_val /= total_batch
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    
    test_acc = sess.run(accuracy, feed_dict={X: X_test_std, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 8.9191, val_loss : 6.3997, acc : 0.500, val_acc : 0.792
Epoch 1, loss : 0.0840, val_loss : 2.3057, acc : 1.000, val_acc : 0.833
Epoch 2, loss : 0.0000, val_loss : 2.6112, acc : 1.000, val_acc : 0.833
Epoch 3, loss : 0.0000, val_loss : 2.7485, acc : 1.000, val_acc : 0.833
Epoch 4, loss : 0.0000, val_loss : 2.8472, acc : 1.000, val_acc : 0.833
Epoch 5, loss : 0.0000, val_loss : 1.7583, acc : 1.000, val_acc : 0.833
Epoch 6, loss : 0.0000, val_loss : 2.6415, acc : 1.000, val_acc : 0.833
Epoch 7, loss : 0.0000, val_loss : 1.8713, acc : 1.000, val_acc : 0.833
Epoch 8, loss : 0.0000, val_loss : 2.2661, acc : 1.000, val_acc : 0.833
Epoch 9, loss : 0.0000, val_loss : 1.6403, acc : 1.000, val_acc : 0.833
test_acc : 1.000


- epoch毎にlossが減少、accが向上しており、学習できているようだ

### 【問題4】House Pricesのモデルを作成  
回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。  
この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使ってください。説明変数はさらに増やしても構いません。  
分類問題と回帰問題の違いを考慮してください。

In [3]:
# データセットの読み込み
dataset_path ="train.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
y = df["SalePrice"]
X = df.loc[:, ["GrLivArea", "YearBuilt"]]
y = np.array(y)[:, np.newaxis]
X = np.array(X)
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# 標準化
sc_x = StandardScaler()
sc_y = StandardScaler()
# トレーニングデータから平均・標準偏差を計算
sc_x.fit(X_train)
sc_y.fit(y_train)
# 上記を用いて標準化
X_train_std = sc_x.transform(X_train)
X_val_std = sc_x.transform(X_val)
X_test_std = sc_x.transform(X_test)
y_train_std = sc_y.transform(y_train)
y_val_std = sc_y.transform(y_val)
y_test_std = sc_y.transform(y_test)


# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 20
num_epochs = 20
n_hidden1 = 20
n_hidden2 = 10
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, 1])
#trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train_std, y_train_std, batch_size=batch_size)

# 計算グラフの構築
def example_net_2(x):
    """
    単純な3層ニューラルネットワーク(HomePrices用)
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
pred = example_net_2(X)
# 目的関数
loss_op = tf.reduce_mean(tf.square(Y - pred))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss=loss_op)
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_mse = 0
        total_mse_val = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            _, mse = sess.run([train_op, loss_op], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            val_mse = sess.run(loss_op, feed_dict={X: X_val_std, Y: y_val_std})
            total_mse += loss
            total_mse_val += val_loss
        total_mse /= total_batch
        total_mse_val /= total_batch
        print("Epoch {}, mse : {:.4f}, val_mse : {:.4f}".format(epoch, mse, val_mse))
    test_mse = sess.run(loss_op, feed_dict={X: X_test_std, Y: y_test_std})
    print("test_mse : {:.3f}".format(test_mse))

Epoch 0, mse : 3.0744, val_mse : 5.2352
Epoch 1, mse : 0.7248, val_mse : 1.2031
Epoch 2, mse : 0.5653, val_mse : 0.6285
Epoch 3, mse : 0.5642, val_mse : 0.5112
Epoch 4, mse : 0.4500, val_mse : 0.4510
Epoch 5, mse : 0.3925, val_mse : 0.4150
Epoch 6, mse : 0.3558, val_mse : 0.3874
Epoch 7, mse : 0.3164, val_mse : 0.3638
Epoch 8, mse : 0.3050, val_mse : 0.3462
Epoch 9, mse : 0.2933, val_mse : 0.3359
Epoch 10, mse : 0.2889, val_mse : 0.3310
Epoch 11, mse : 0.2792, val_mse : 0.3217
Epoch 12, mse : 0.2759, val_mse : 0.3180
Epoch 13, mse : 0.2723, val_mse : 0.3115
Epoch 14, mse : 0.2697, val_mse : 0.3082
Epoch 15, mse : 0.2678, val_mse : 0.3043
Epoch 16, mse : 0.2656, val_mse : 0.2999
Epoch 17, mse : 0.2639, val_mse : 0.2952
Epoch 18, mse : 0.2643, val_mse : 0.2928
Epoch 19, mse : 0.2657, val_mse : 0.2897
test_mse : 0.404


- 説明・目的変数ともに標準化し、出力ノード数を1にしてニューラルネットワークを回帰問題に適応した
- 学習は進んでいるが、train/validationと比較してtestの $MSE$ が大きく、Varianceが高いモデルで過学習しているようだ

### 【問題5】MNISTのモデルを作成  
ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。


3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。


スクラッチで実装したモデルの再現を目指してください。



In [6]:
from keras.datasets import mnist

# データセットのロード、確認
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# 平滑化
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)
# 前処理
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255
# yをone_hotへ
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train = enc.fit_transform(y_train[:,None])
y_test = enc.fit_transform(y_test[:,None])
# trainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 20
num_epochs = 5
n_hidden1 = 40
n_hidden2 = 20
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
#trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

# 計算グラフの構築
def example_net_3(x):
    """
    単純な3層ニューラルネットワーク(MNIST用)
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net_3(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss=loss_op)
# 推定結果
correct_pred = tf.equal(tf.argmax(Y, axis=1), tf.argmax(logits, axis=1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        total_loss_val = 0
        total_acc_val = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            _, loss, acc = sess.run([train_op, loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
            total_loss += loss
            total_acc += acc
            total_loss_val += val_loss
            total_acc_val += val_acc
        total_loss /= total_batch
        total_acc /= total_batch
        total_loss_val /= total_batch
        total_acc_val /= total_batch
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 1.6213, val_loss : 1.3359, acc : 0.350, val_acc : 0.511
Epoch 1, loss : 0.5297, val_loss : 0.5583, acc : 0.800, val_acc : 0.851
Epoch 2, loss : 0.6392, val_loss : 0.3210, acc : 0.750, val_acc : 0.917
Epoch 3, loss : 0.5376, val_loss : 0.2981, acc : 0.800, val_acc : 0.920
Epoch 4, loss : 0.2899, val_loss : 0.2576, acc : 0.850, val_acc : 0.934
test_acc : 0.932


- 全結合層のみのモデル（隠れ層：2層、ノード数：40,20）を構築した
- Epoch毎に学習は進み、スクラッチ時と同程度の性能は出ている